In [1]:
def preprocess(file_name):
    with open(file_name) as f:
        d = json.load(f)
    data = json_normalize(d['summaries'])

    get_names = lambda x: [i['name'] for i in x if i['name'] is not ""] if len(x) else []
    data['architectures'] = data['architectures'].map(get_names)
    data['architectures'] = data['architectures'].apply(sorted)
    data['num_archs'] = data['architectures'].map(len)

    data['architectures'] = data['architectures'].map(lambda x: ", ".join(x))
    data = data.drop(['certification_status', 'id',
                      'filter_type', 'logo_url.large',
                      'logo_url.small', 'popularity',
                      'publisher.id', 'name', 'publisher.name', 'type'], axis=1)
    labels_as_list = lambda x: ", ".join(i['label'] for i in x if i['label'] is not "") if \
        len(x) else None
    data['operating_systems'] = data['operating_systems'].map(labels_as_list)

    # def to_numbers(x):
    #     no_plus = str(x)[:-1] if "+" in str(x) else x
    #     print(no_plus)
    #     print(x)
    #     no_plus = no_plus.strip()
    #     number, suffix = float(no_plus[:-1]), no_plus[-1]
    #
    #     if suffix is "K":
    #         return number * 1000
    #     elif suffix is "M":
    #         return number * 1000000
    #     else:
    #         return number
    # data['pull_count'] = data['pull_count'].map(to_numbers)
    data['pull_count'] = data['pull_count'].map(lambda x: str(x)[:-1] if "+" in str(x) else str(x))
    data['pull_count'] = data['pull_count'].map(
        lambda x: float(x[:-1]) * 1000 if x[-1] is "K" else\
        float(x[:-1]) * 1000000 if x[-1] is "M" else float(x))

    # data['categories_exists'] = data['categories'].map(lambda x: 1 if x else 0)
    data['category_count'] = data['categories'].map(lambda x: len(x) if x else 0)
    data = data.drop(['categories'], axis=1)
    data.set_index('slug')
    return data

In [6]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [7]:
DATA_DIR = "../data/recent-data/"
import os
files = sorted(os.listdir(DATA_DIR), reverse=True)
# for i in range(len(files) - 1):
file_1 = os.path.join(DATA_DIR, files[0])
file_2 = os.path.join(DATA_DIR, files[1])
print(file_1, file_2)
df1, df2 = preprocess(file_1), preprocess(file_2)

../data/recent-data/recent_30-04__20.json ../data/recent-data/recent_30-04__16.json


In [23]:
merged = pd.merge(df1[['slug', 'pull_count']], df2[['slug', 'pull_count']], on='slug')

In [28]:
df1.loc[df1['slug'] == 'kaazing-gateway']

,architectures,created_at,operating_systems,pull_count,short_description,slug,source,star_count,updated_at,num_archs,category_count
276,"amd64, arm64",2016-06-16T00:00:00Z,Linux,NaN,Official build of Kaazing Gateway.,kaazing-gateway,store,0,2019-04-30T19:06:49.146955Z,2,1


In [24]:
merged.loc[merged['pull_count_x'] != merged['pull_count_y']]

,slug,pull_count_x,pull_count_y
0,liquidweb/php-fpm_exporter,7.0,1.0
1,jgoerzen/home-assistant,226.0,222.0
4,cadster/home-assistant-docker,219.0,215.0
6,decinho13/base-notebook,849.0,811.0
8,ascensive/irc-rss-feed-bot,643.0,639.0
9,sanfamite/home-assistant-mychevy,540.0,536.0
10,mooseh81/homeassistant,5700.0,5600.0
15,nwykes/home-assistant-docker,218.0,214.0
17,rooster791/docker-hass,585.0,578.0
18,pschmitt/hass-slave,736.0,732.0


In [ ]:
file_3 = os.path.join(DATA_DIR, files[2])